## Capture and save stereo images to be used for calibration later ##

<b>Description</b> The following code is meant to display a live feed of the two usb cameras and automatically capture and store a set amount of stereo photos whenever a chessboard is detected. A sleep period of 5 seconds is set in between photos.
<hr>

#### Cell Block 1 ####
<b>Description:</b> Initializes the cameras and sets them to running.<br>

In [ ]:
# USB Camera (Logitech C270 webcam)
# From above input, set "capture_device" variable to the number at the end of "/dev/video_"
from jetcam.usb_camera import USBCamera
camWidth=640
camHeight=480
cameraR = USBCamera(width=camWidth, height=camHeight, capture_device=0) # confirm the capture_device number
cameraL = USBCamera(width=camWidth, height=camHeight, capture_device=1)

imageR = cameraR.read()
imageL = cameraL.read()

cameraR.running = True
cameraL.running = True
print("camera created")

#### Cell Block 2 ####
<b>Description:</b> The code below creates the widgets to display the live feed of the cameras within the Jupyter Notebook. A counter for images taken is also provided to keep track.

In [ ]:
import ipywidgets
import os
from IPython.display import display, HTML
from jetcam.utils import bgr8_to_jpeg
import traitlets

image_widgetR = ipywidgets.Image(format='jpeg')
image_widgetR.value = bgr8_to_jpeg(imageR)

image_widgetL = ipywidgets.Image(format='jpeg')
image_widgetL.value = bgr8_to_jpeg(imageL)

button_layout = ipywidgets.Layout(width='128px', height='64px')

camera_linkR = traitlets.dlink((cameraR, 'value'), (image_widgetR, 'value'), transform=bgr8_to_jpeg)
camera_linkL = traitlets.dlink((cameraL, 'value'), (image_widgetL, 'value'), transform=bgr8_to_jpeg)

pic_count = ipywidgets.IntText(layout=button_layout, value=len(os.listdir('dataset')))

display(ipywidgets.HBox([image_widgetR, image_widgetL]))
display(ipywidgets.HBox([pic_count]))  

#### Cell Block 3 ####
<b>Description:</b> The code below will automatically save the frames from both cameras the instant a chessboard is detected in them. The code will then wait for 5 seconds before proceeding to take another pair of photos. The photos are saved in a directory named dataset.<br>
<b>Note:</b> <i>I did not make the code check if the directory exists or not. Please create the directory before running the code.</i>

In [ ]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt

while (len(os.listdir('dataset'))/2) < 50:
    directory = "dataset"
    cameraL_frame = cameraL.value
    cameraR_frame = cameraR.value
    grayL = cv2.cvtColor(cameraL_frame,cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(cameraR_frame,cv2.COLOR_BGR2GRAY)
    retR, cornersR = cv2.findChessboardCorners(grayR, (9,6),None)
    retL, cornersL = cv2.findChessboardCorners(grayL, (9,6),None)
    # If found, add object points, image points (after refining them)
    if retL == True and retR == True:
        print("Found Chessboard")
        counter = int(len(os.listdir('dataset'))/2)
        image_pathL = os.path.join(directory, str(counter) + '_Left.jpg')
        image_pathR = os.path.join(directory, str(counter) + '_Right.jpg')
        cv2.imwrite(image_pathL, cameraL_frame)
        cv2.imwrite(image_pathR, cameraR_frame)
        pic_count.value = (len(os.listdir('dataset'))/2)+0.5
        cv2.waitKey(5000)

print("Pictures have been taken")
    
    